In [1]:
import requests
import json
import time
from bs4 import BeautifulSoup
import pandas as pd
import datetime

import plotly.express as px
import plotly.graph_objects as go

# コンテスト情報

In [2]:
contest_id = "rco-contest-2019-qual"
problem_id = "rco_contest_2019_qual_b"

contest_start = datetime.datetime(2023,1,13,19,0,0)
contest_end = datetime.datetime(2023,1,22,19,0,0)


# データ作成
コンテスト期間中の提出結果から、リストを作成

In [3]:
df = pd.read_csv("submissions.csv", index_col = 0)
df.columns = ["名前","チーム名", "言語", "スコア", "提出日時", "実行時間"]
df.head()

,名前,チーム名,言語,スコア,提出日時,実行時間
0,r3yohei,benchmark,Rust (1.42.0),607852,2023-01-22 18:55:38,1991 ms
1,brthyyjp,benchmark,PyPy3 (7.3.0),595747,2023-01-22 18:55:27,1901 ms
2,fujikawahiroaki,逆襲のバベルクライマーズ,Crystal (0.33.0),565300,2023-01-22 18:51:48,1963 ms
3,r3yohei,benchmark,Rust (1.42.0),611391,2023-01-22 18:50:36,1991 ms
4,brthyyjp,benchmark,PyPy3 (7.3.0),597179,2023-01-22 18:50:14,1900 ms


## 個人ランキング

In [4]:
df_individual = df.loc[df.groupby("名前")["スコア"].idxmax()]
df_individual = df_individual.drop(columns = ["言語", "提出日時", "実行時間"])
df_individual.sort_values("スコア", ascending=False, inplace=True)
df_individual.reset_index(inplace=True, drop=True)
df_individual["順位"] = range(1,len(df_individual)+1)
df_individual = df_individual.reindex(columns = ["順位", "名前", "チーム名", "スコア"]) 
df_individual.head()

,順位,名前,チーム名,スコア
0,1,cplusplusonly,>euphonic,652081
1,2,highjump,山走暁飛,647497
2,3,zatt,霧ヶ峰めっちゃ良い買い物しました！！！！,621224
3,4,hiraku,>euphonic,620430
4,5,8wings,山走暁飛,615680


## チームランキング

In [5]:
df_team = df.loc[df.groupby("チーム名")["スコア"].idxmax()]
df_team = df_team.drop(columns = ["言語", "提出日時", "実行時間"])
df_team = df_team.reindex(columns = ["チーム名", "名前", "スコア"])
df_team.sort_values("スコア", ascending = False, inplace = True)
df_team.reset_index(inplace = True, drop = True)
df_team["順位"] = range(1,len(df_team)+1)
df_team = df_team.reindex(columns = ["順位", "チーム名", "名前", "スコア"]) 
df_team.head()

,順位,チーム名,名前,スコア
0,1,>euphonic,cplusplusonly,652081
1,2,山走暁飛,highjump,647497
2,3,霧ヶ峰めっちゃ良い買い物しました！！！！,zatt,621224
3,4,benchmark,r3yohei,611568
4,5,輝かしい未来を目指す不屈の戦い人,akane831,594863


# グラフ作成

1.   縦軸：スコア、横軸：時間の折れ線グラフ（個人）
2.   縦軸：スコア、横軸：時間の折れ線グラフ（チーム）
3.   言語ごとのランキング
4.   スコアとレートの相関関係（アルゴ、ヒューリスティック）




In [6]:
df_line_indv = df.sort_values("提出日時", ascending = True)
df_line_indv["スコア"] = df_line_indv.groupby("名前")["スコア"].cummax()

fig_individual = px.line(df_line_indv , x="提出日時", y="スコア", color="名前")
fig_individual.show()

In [7]:
df_line_team = df.sort_values("提出日時", ascending = True)
df_line_team["スコア"] = df_line_team.groupby("チーム名")["スコア"].cummax()

fig_team = px.line(df_line_team, x="提出日時", y="スコア", color="チーム名")
fig_team.show()

In [8]:
df_lang = df.replace("C++ (GCC 9.2.1)", "C++"
            ).replace("C++ (Clang 10.0.0)", "C++"
            ).replace("C++14", "C++"
            ).replace("C# (.NET Core 3.1.201)", "C#"
            ).replace("PyPy3 (7.3.0)", "Python"
            ).replace("Python (3.8.2)", "Python"
            ).replace("Rust (1.42.0)", "Rust"
            ).replace("Crystal (0.33.0)", "Crystal"
            ).replace("D (DMD 2.091.0)", "D"
            ).replace("D (LDC 1.20.1)", "D"
            ).replace("Common Lisp (SBCL 2.0.3)", "Common Lisp"
            ).replace("Java (OpenJDK 11.0.6)", "Java"
            )
df_lang = df_lang.loc[df_lang.groupby("名前")["スコア"].idxmax()]

fig_lang = px.box(df_lang, x="言語", y="スコア", points='all')
fig_lang.show()

In [9]:
"""
df_rate = df.loc[df.groupby("名前")["スコア"].idxmax()]

layout = go.Layout(
    xaxis = dict(title="レート", range = [0,3001], dtick=500),   # rangeで範囲、dtick で区間幅
    yaxis = dict(title="スコア（対数）"))

fig_rate = go.Figure(layout = layout)
fig_rate.add_trace(go.Scatter(x = df_rate["アルゴレート"],
                              y = df_rate["スコア"],
                              mode = "markers",
                              name = "アルゴ"))
fig_rate.add_trace(go.Scatter(x = df_rate["ヒューリスティックレート"],
                              y = df_rate["スコア"],
                              mode = "markers",
                              name = "ヒューリスティック"))
fig_rate.update_yaxes(type="log")
fig_rate.show()
"""

'\ndf_rate = df.loc[df.groupby("名前")["スコア"].idxmax()]\n\nlayout = go.Layout(\n    xaxis = dict(title="レート", range = [0,3001], dtick=500),   # rangeで範囲、dtick で区間幅\n    yaxis = dict(title="スコア（対数）"))\n\nfig_rate = go.Figure(layout = layout)\nfig_rate.add_trace(go.Scatter(x = df_rate["アルゴレート"],\n                              y = df_rate["スコア"],\n                              mode = "markers",\n                              name = "アルゴ"))\nfig_rate.add_trace(go.Scatter(x = df_rate["ヒューリスティックレート"],\n                              y = df_rate["スコア"],\n                              mode = "markers",\n                              name = "ヒューリスティック"))\nfig_rate.update_yaxes(type="log")\nfig_rate.show()\n'

## HTMLで書き出し

In [10]:
fig_individual.write_html("line_individual.html")
fig_team.write_html("line_team.html")
fig_lang.write_html("box_lang.html")
#fig_rate.write_html("scatter_rate.html")

In [11]:
df_individual.to_html(border = 0,
                      classes = ["table", "table-striped", "table-hover"],
                      index = False,
                      justify = "left")

'<table class="dataframe table table-striped table-hover">\n  <thead>\n    <tr style="text-align: left;">\n      <th>順位</th>\n      <th>名前</th>\n      <th>チーム名</th>\n      <th>スコア</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <td>1</td>\n      <td>cplusplusonly</td>\n      <td>&gt;euphonic</td>\n      <td>652081</td>\n    </tr>\n    <tr>\n      <td>2</td>\n      <td>highjump</td>\n      <td>山走暁飛</td>\n      <td>647497</td>\n    </tr>\n    <tr>\n      <td>3</td>\n      <td>zatt</td>\n      <td>霧ヶ峰めっちゃ良い買い物しました！！！！</td>\n      <td>621224</td>\n    </tr>\n    <tr>\n      <td>4</td>\n      <td>hiraku</td>\n      <td>&gt;euphonic</td>\n      <td>620430</td>\n    </tr>\n    <tr>\n      <td>5</td>\n      <td>8wings</td>\n      <td>山走暁飛</td>\n      <td>615680</td>\n    </tr>\n    <tr>\n      <td>6</td>\n      <td>r3yohei</td>\n      <td>benchmark</td>\n      <td>611568</td>\n    </tr>\n    <tr>\n      <td>7</td>\n      <td>brthyyjp</td>\n      <td>benchmark</td>\n      <td>597179</td

In [12]:
df_team.to_html(border = 0,
                classes = ["table", "table-striped", "table-hover"],
                index = False,
                justify = "left")

'<table class="dataframe table table-striped table-hover">\n  <thead>\n    <tr style="text-align: left;">\n      <th>順位</th>\n      <th>チーム名</th>\n      <th>名前</th>\n      <th>スコア</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <td>1</td>\n      <td>&gt;euphonic</td>\n      <td>cplusplusonly</td>\n      <td>652081</td>\n    </tr>\n    <tr>\n      <td>2</td>\n      <td>山走暁飛</td>\n      <td>highjump</td>\n      <td>647497</td>\n    </tr>\n    <tr>\n      <td>3</td>\n      <td>霧ヶ峰めっちゃ良い買い物しました！！！！</td>\n      <td>zatt</td>\n      <td>621224</td>\n    </tr>\n    <tr>\n      <td>4</td>\n      <td>benchmark</td>\n      <td>r3yohei</td>\n      <td>611568</td>\n    </tr>\n    <tr>\n      <td>5</td>\n      <td>輝かしい未来を目指す不屈の戦い人</td>\n      <td>akane831</td>\n      <td>594863</td>\n    </tr>\n    <tr>\n      <td>6</td>\n      <td>運営</td>\n      <td>ainem</td>\n      <td>593008</td>\n    </tr>\n    <tr>\n      <td>7</td>\n      <td>逆襲のバベルクライマーズ</td>\n      <td>yuuki_n</td>\n      <td>57784